In [1]:
from __future__ import absolute_import, division, print_function

import os

# !pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-alpha0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [14]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
checkpoint_path = 'traninig_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [16]:
# Create chekpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)


In [17]:
model = create_model()

In [18]:
model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 768/1000 [======================>.......] - ETA: 0s - loss: 1.2813 - accuracy: 0.6589
Epoch 00001: saving model to traninig_1/cp.ckpt
1000/1000 [==============================] - 0s 301us/sample - loss: 1.1302 - accuracy: 0.6900 - val_loss: 0.6933 - val_accuracy: 0.7940
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4020 - accuracy: 0.8919
Epoch 00002: saving model to traninig_1/cp.ckpt
1000/1000 [==============================] - 0s 114us/sample - loss: 0.4087 - accuracy: 0.8880 - val_loss: 0.5176 - val_accuracy: 0.8320
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.3010 - accuracy: 0.9193
Epoch 00003: saving model to traninig_1/cp.ckpt
1000/1000 [==============================] - 0s 113us/sample - loss: 0.2957 - accuracy: 0.9180 - val_loss: 0.4720 - val_accuracy: 0.8500
Epoch 4/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.2152 - accuracy: 0.9505
Epoch

In [19]:
model = create_model()

In [20]:
loss, acc = model.evaluate(test_images, test_labels)

1000/1000 [==============================] - 0s 73us/sample - loss: 2.3397 - accuracy: 0.0850


In [21]:
print('Untranied model accuracy: {:5.2f}%'.format(100*acc))

Untranied model accuracy:  8.50%


In [22]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 33us/sample - loss: 0.4035 - accuracy: 0.8740
Restored model accuracy: 87.40%


In [23]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [24]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, verbose=1, save_weights_only=True,
                period=5)

In [25]:
model = create_model()

In [26]:
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback],
         validation_data=(test_images, test_labels), 
         verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [27]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [28]:
model = create_model()


In [29]:
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 69us/sample - loss: 0.4952 - accuracy: 0.8750
Restored model, accuracy: 87.50%


In [30]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 66us/sample - loss: 0.4952 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## Save the entire model

The model and optimizer can be saved to a file that contains both their state (weights and variables), and the model configuration. This allows you to export a model so it can be used without access to the original python code. Since the optimizer-state is recovered you can even resume training from exactly where you left off.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (HDF5, Saved Model) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (HDF5, Saved Model)

## As an HDF5 file
Keras provides a basic save format using the HDF5 standard. For our purposes, the saved model can be treated as a single binary blob.

In [32]:
model = create_model()

In [34]:
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 91us/sample - loss: 0.4311 - accuracy: 0.8910
Epoch 2/5
1000/1000 [==============================] - 0s 78us/sample - loss: 0.3032 - accuracy: 0.9170
Epoch 3/5
1000/1000 [==============================] - 0s 78us/sample - loss: 0.2105 - accuracy: 0.9490
Epoch 4/5
1000/1000 [==============================] - 0s 74us/sample - loss: 0.1658 - accuracy: 0.9630
Epoch 5/5
1000/1000 [==============================] - 0s 74us/sample - loss: 0.1289 - accuracy: 0.9750


In [35]:
# Save the mode to HDF5 format
model.save('my_model.h5')

In [36]:
new_model = keras.models.load_model('my_model.h5')

In [37]:
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [38]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 72us/sample - loss: 0.4037 - accuracy: 0.8760
Restored model, accuracy: 87.60%


In [39]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 146us/sample - loss: 1.1730 - accuracy: 0.6440
Epoch 2/5
1000/1000 [==============================] - 0s 76us/sample - loss: 0.4288 - accuracy: 0.8760
Epoch 3/5
1000/1000 [==============================] - 0s 82us/sample - loss: 0.2825 - accuracy: 0.9330
Epoch 4/5
1000/1000 [==============================] - 0s 73us/sample - loss: 0.2036 - accuracy: 0.9550
Epoch 5/5
1000/1000 [==============================] - 0s 72us/sample - loss: 0.1467 - accuracy: 0.9710


In [40]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))


In [41]:
tf.keras.experimental.export_saved_model(model, saved_model_path)

W0314 11:36:05.422201 140321221056320 deprecation.py:323] From /home/rohan/miniconda3/envs/deep_learning_tf/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0314 11:36:05.423016 140321221056320 tf_logging.py:161] Export includes no default signature!
W0314 11:36:05.731136 140321221056320 tf_logging.py:161] Export includes no default signature!


In [42]:
saved_model_path

'./saved_models/1552588537'

In [43]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.predict(test_images).shape

(1000, 10)

This technique saves everything:

- The weight values
- The model's configuration(architecture)
- The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.



In [45]:
# The model has to be complied before evaluation
new_model.compile(optimizer=model.optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

In [47]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 35us/sample - loss: 0.4550 - accuracy: 0.8470
Restored model, accuracy: 84.70%
